In [1]:
import uuid
import asyncio
import pandas as pd
from tqdm import tqdm
from order_book_simulator.config import OrderRequest, Priority
from order_book_simulator.config import OrderRequest, Side, Priority
from order_book_simulator.core import OrderBook, MatchingEngine
from order_book_simulator.websocket import ConnectionManager
from order_book_simulator.analysis import MetricsCalculator

In [2]:
orders_df = pd.read_csv('../data/orders-1.csv')

In [3]:
# just to adhere to my definition of the OrdrerRequest dataclass
# unix timestamp is more efficient for computation and storage
# trader_uuid is more secure and flexible than group number
# (in real life, traders won't identify themselves with group numbers)
# so we convert them here
orders_df["unix_timestamp"] = pd.to_datetime(orders_df["Timestamp"]).astype(int) // 10**9
group_to_uuid = {group: uuid.uuid4() for group in orders_df["Group Number"].unique()}
orders_df["trader_uuid"] = orders_df["Group Number"].map(group_to_uuid)

In [4]:
order_book = OrderBook()
metrics_calculator = MetricsCalculator(order_book, 0.01)
connection_manager = ConnectionManager()

# websocket related stuff are not required for this assignment but we designed some functions to allow that
matching_engine = MatchingEngine(order_book, 
                                 connection_manager, # not needed for assignment
                                 asyncio.get_event_loop(), # not needed for assignment
                                 metrics_calculator)

In [5]:
filled_trades = []
total_orders = len(orders_df)
checkpoints = {4, 19, total_orders - 1} 

for index, row in tqdm(orders_df.iterrows(), total=total_orders, desc="Processing orders"):
    order_request = OrderRequest(
        trader_id = row["trader_uuid"],
        side = row["Buy Sell Direction"],
        priority=Priority.HIGH,
        timestamp=row["unix_timestamp"],
        price=row["Order Price"],
        quantity=row["Quantity"]
    )
    trade = matching_engine.process_order(order_request)
    if trade:
        filled_trades.extend(trade)
    
    if index in checkpoints:
        label = "FINAL" if index == total_orders - 1 else f"#{index + 1}"
        print(f"\n=== Order Book after {label} orders ===")
        print(order_book)

Processing orders:   0%|          | 0/248 [00:00<?, ?it/s]


=== Order Book after #5 orders ===


┏━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Bid Price ┃ Bid Quantity ┃ Ask Price ┃ Ask Quantity ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│     11.00 │           20 │     12.00 │           30 │
│           │              │     50.00 │           11 │
└───────────┴──────────────┴───────────┴──────────────┘



=== Order Book after #20 orders ===


┏━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Bid Price ┃ Bid Quantity ┃ Ask Price ┃ Ask Quantity ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│     11.00 │           50 │     12.00 │           55 │
│     10.00 │           75 │     50.00 │           11 │
└───────────┴──────────────┴───────────┴──────────────┘



=== Order Book after FINAL orders ===


┏━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Bid Price ┃ Bid Quantity ┃ Ask Price ┃ Ask Quantity ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│     10.00 │          190 │     11.00 │           81 │
│      9.00 │          228 │     12.00 │          510 │
│      8.00 │           25 │     13.00 │          155 │
│      7.00 │          130 │     15.00 │           50 │
│           │              │     50.00 │           11 │
└───────────┴──────────────┴───────────┴──────────────┘

Processing orders: 100%|██████████| 248/248 [00:00<00:00, 9524.91it/s]

In [7]:
final_metrics = metrics_calculator.calculate_averages()

--- Final Metrics ---

Average Bid-Ask Spread (in Ticks): 123.1373

Average Bid-Ask Spread (in Basis Points): 1233.6630

Average Best-Level Market Depth (in Dollars): $2,939.49